A.S. Lundervold, 25.10.2021

# Introduction

When faced with a predictive task based on non-imaging data, an approach that has seen some success in certain cases, and therefore worthwhile knowing about and considering, is to find a way to **represent your data as images**. 

This makes it possible to use the relatively well-developed image analysis techniques in deep learning, f.ex. the landscape surrounding convolutional neural networks. 

Of course, this clearly won't make sense in _all_ cases. It can be hard--or impossible--to capture the information needed to make useful predictions in an image representation, in a way that's adapted to the image-based predictive models you'll want to use. 

Still, there are examples of this leading to good results:

An approach to **sound classification**, as used f.ex. for speech recognition, is based on turning audio into _spectrograms_, e.g. using mel spectrograms to represent speech: 

<img width=60% src="assets/melspecs.png">

[Catching fraudsters using their mouse movements](https://www.splunk.com/en_us/blog/security/deep-learning-with-splunk-and-tensorflow-for-security-catching-the-fraudster-in-neural-networks-with-behavioral-biometrics.html) is another interesting example. So is this one: [malware classification](https://ieeexplore.ieee.org/abstract/document/8328749). 

An example of the more general idea of representing **time series data to image data** from my own work is a way to represent **longitudinal measurements** recorded from multiple dementia subjects as greyscale images, one per subject:

<img width=70% src="assets/long_images1.png">

<img src="assets/long_images2.png">

This notebook illustrates the approach on two quite different data sets and problems: 

1. Audio recordings and sound classification (this notebook)
2. Small molecules and **prediction of bioactivity for given targets**, a core component of **drug discovery** (the next notebook)

This is to get you thinking about whether transforming data into images and use image-based deep learning models could be useful in your own work. 

# Setup

In [ ]:
# Note: you may disregard these. Added to deal with PyTorch and CUDA version 
# issues on my computer
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import torch
torch.cuda.get_device_name(0)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory, as that makes some difference for the code below.
# We'll do this in every notebook of the course.
if 'google.colab' in str(get_ipython()):
    print('The notebook is running on Colab. colab=True.')
    colab=True
else:
    print('The notebook is not running on Colab. colab=False.')
    colab=False

In [ ]:
# Set to True if you're using Paperspace Gradient:
gradient=False

In [ ]:
if colab or gradient:
    !pip install -Uqq fastbook
    !pip install -Uqq fastaudio
    import fastbook
    fastbook.setup_book()
    from fastbook import *
    from fastai.vision.all import *
    NB_DIR = Path.cwd()
else:
    from fastai.vision.all import *
    NB_DIR = Path.cwd()
    DATA = Path('/data2/alex/pcs-data')  # Set this to where you want to store downloaded data  
    
if colab:
    DATA = Path('./gdrive/MyDrive/ColabData')
    DATA.mkdir(exist_ok=True)
if gradient:
    DATA = Path('/storage')
    DATA.mkdir(exist_ok=True)

In [ ]:
import os, shutil, gc

# Sound classification

<img src="assets/ESC-50.png">

https://github.com/karolpiczak/ESC-50

# Get the data

_Note: the full data set is 600MB. You may want to use the sample data instead_. If you set `sample=True` below you will use a version of the ESC-50 data set that only has nine classes. 

In [ ]:
sample = False

In [ ]:
if not colab:

    if sample:
        path = untar_data('https://www.dropbox.com/s/iuq72ty8bu1v5nc/ecs50-sample.zip?dl=1', 
                          fname = 'data/ecs50-sample.zip', dest=DATA)
    if not sample:
        path = untar_data('https://github.com/karoldvl/ESC-50/archive/master.zip', 
                          fname=DATA/'master.zip', dest=DATA)
        
if colab:
    if sample:
        path = untar_data('https://www.dropbox.com/s/iuq72ty8bu1v5nc/ecs50-sample.zip?dl=1')
    if not sample:
        path = untar_data('https://github.com/karoldvl/ESC-50/archive/master.zip')

In [ ]:
path.ls()

In [ ]:
audio = path/'audio'
if sample:
    df = pd.read_csv(path/'meta'/'esc50-sample.csv')
if not sample:
    df = pd.read_csv(path/'meta'/'esc50.csv')

In [ ]:
audio.ls()

In [ ]:
df.head()

In [ ]:
df.category.value_counts()

# Explore the data 

The audio files are stored as WAV files. We can work with audio files in Python using the very convenient `librosa` library. 

<img src="https://librosa.org/images/librosa_logo_text.png">

> Below, we'll use the `fastaudio` library, which relies heavily on librosa. 

In [ ]:
import librosa

In [ ]:
example_fn = audio.ls()[0]
example_fn

We can load the WAV file and store the waveform in `y` and the sampling rate in `sr`:

In [ ]:
y, sr = librosa.load(example_fn)

`y` is then just a Numpy array, and we can work with it as such:

In [ ]:
y

`sr` is a number (extracted from the WAV file):

In [ ]:
sr

Here's a plot of `y`:

In [ ]:
plt.plot(y)
plt.show()

Using the sampling rate we can put the time on the x-axis. This is implemented by librosa's `waveplot`:

In [ ]:
from librosa.display import waveplot

In [ ]:
waveplot(y, sr)
plt.show()

Here's a bunch of examples:

In [ ]:
for fn in df.sample(n=5).filename:
    y, sr = librosa.load(audio/fn)
    waveplot(y, sr)
    plt.show()

We can play the audio files via our notebooks using IPython:

In [ ]:
from IPython.display import Audio

In [ ]:
Audio(example_fn)

In [ ]:
def plot_play_clip(a, sr=None):
    if type(a)!=np.ndarray:
        print(f"File {a.stem}")
        print(f"Category: {df.loc[df.filename==f'{a.stem}.wav'].category.values}")
        a, sr = librosa.load(a, sr=sr)
    waveplot(a, sr)
    plt.show()
    return Audio(a, rate=sr)

In [ ]:
plot_play_clip(audio.ls()[0])

In [ ]:
plot_play_clip(audio.ls()[1])

In [ ]:
plot_play_clip(audio.ls()[2])

In [ ]:
plot_play_clip(audio.ls()[3])

We can try to get a sense of the variation in the data by listening to some examples from specific categories:

In [ ]:
def random_category_examples(c, k=3):
    relevant = df.loc[df.category==c]
    for i in range(k):
        fn = relevant.iloc[i].filename
    return relevant.sample(n=k)

In [ ]:
df.category.value_counts()

In [ ]:
examples = random_category_examples('crow')
examples

In [ ]:
plot_play_clip(audio/examples.iloc[0].filename)

In [ ]:
plot_play_clip(audio/examples.iloc[1].filename)

In [ ]:
plot_play_clip(audio/examples.iloc[2].filename)

All the audio files are 5 seconds long:

In [ ]:
librosa.get_duration(y, sr)

> This simplifies some things for us. If you're dealing with time series of different lengths you may consider cropping the series or splitting it up in parts. 

# Side-note: Audio as time series

> These audio recordings are examples of (univariate) time series and can be studied and dealt with using what you learned about in time series analysis. 

## (Re)sampling

In [ ]:
plot_play_clip(example_fn, sr=None)

In [ ]:
plot_play_clip(example_fn, sr=1000)

> All the various techniques and issues related to sampling of continuous signals applies. E.g. the [Nyquist-Shannon sampling theorem](https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem) stating requirements and limitations for accurate capture of a signal.

## Combining time series

In [ ]:
list(examples.filename)

In [ ]:
ex1 = audio/list(examples.filename)[0]
ex2 = audio/list(examples.filename)[1]

In [ ]:
ex1

In [ ]:
ex2

In [ ]:
plot_play_clip(ex1)

In [ ]:
plot_play_clip(ex2)

In [ ]:
y1, sr1 = librosa.load(ex1)
y2, sr2 = librosa.load(ex2)

In [ ]:
y1

In [ ]:
y2

In [ ]:
y = y1 + y2

In [ ]:
plot_play_clip(y, sr=sr1)

## Time-domain versus frequency domain

<img src="https://i.stack.imgur.com/yrMqS.jpg">

As you learned about in the time series module of the course, it is useful to work with time series in the time domain _and_ in the frequency domain. That's true also for audio. 

> From the time domain to the frequency domain and back.

### FFT: The fast fourier transform

In [ ]:
y, sr = librosa.load(example_fn)

In [ ]:
import scipy

In [ ]:
plot_play_clip(y, sr=sr)

In [ ]:
y_fft = scipy.fftpack.fft(y)
y_fft

In [ ]:
plt.plot(abs(y_fft),'r') 
plt.show()

In [ ]:
scipy.fftpack.ifft(y_fft).real

In [ ]:
y

### Short-time fourier transform

In [ ]:
y, sr = librosa.load(example_fn)

In [ ]:
S = np.abs(librosa.stft(y))

In [ ]:
S.shape

In [ ]:
S

In [ ]:
plot_play_clip(example_fn)

In [ ]:
fig, ax = plt.subplots()
img = librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), y_axis='log', x_axis='time', ax=ax)
ax.set_title('Power spectrogram')
fig.colorbar(img, ax=ax, format="%+2.0f dB")
plt.show()

### Wavelets

$$T(a,b) = \frac{1}{\sqrt a} \int_{-\infty}^{\infty} x(t)\psi^* \frac{(t-b)}{a}dt$$

a: dilation parameter<br>
b: location of wavelet<br>
$\psi$: wavelet function<br>
x: signal

So, now we know which frequencies exist in the time signal and where they exist.

This is the advantage that wavelet transform has over FFT. It can capture spectral and temporal information simultaneously.

Morlet wavelet:

$$\psi(t) = \exp(-i\omega_0 t) \exp(-t^2/2) $$

In [ ]:
t = np.linspace(-10.,10.,1000)
w0 = 5

In [ ]:
psi = np.exp(-1j * w0 * t) * np.exp(-t**2 / 2)

In [ ]:
plt.plot(t, psi)
plt.show()

https://github.com/AdityaDutt/Audio-Classification-Using-Wavelet-Transform/blob/main/wavelet_tutorial.py

**Computing wavelet coefficients**

In [ ]:
!{sys.executable} -m pip install PyWavelets scaleogram

In [ ]:
import pywt

In [ ]:
import scaleogram as scg 

In [ ]:
wavelet = 'morl'
widths = np.arange(1, 256)
dt = 1 / sr

In [ ]:
dt

In [ ]:
frequencies = pywt.scale2frequency(wavelet, widths) / dt

In [ ]:
frequencies.shape

In [ ]:
upper = ([x for x in range(len(widths)) if frequencies[x] > 1000])[-1]
lower = ([x for x in range(len(widths)) if frequencies[x] < 80])[0]
widths = widths[upper:lower] # Select scales in this frequency range

In [ ]:
wavelet_coeffs, freqs = pywt.cwt(y, widths, wavelet = wavelet, sampling_period=dt)

In [ ]:
wavelet_coeffs.shape

In [ ]:
t, _ = np.linspace(0, 200, wavelet_coeffs.shape[1], retstep=True)

In [ ]:
wavelet_coeffs.shape

Show some of the scales and some of the coefficients:

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(wavelet_coeffs[:,:800], cmap='coolwarm')
plt.ylabel("Scales")
plt.yticks(widths[0::11])
plt.title("Scalogram")
plt.show()

In [ ]:
def get_wavelet_scalogram(a, wavelet = 'morl', plot_some=True):
    y, sr = librosa.load(a)
    widths = np.arange(1, 256)
    dt = 1 / sr
    frequencies = pywt.scale2frequency(wavelet, widths) / dt
    
    upper = ([x for x in range(len(widths)) if frequencies[x] > 1000])[-1]
    lower = ([x for x in range(len(widths)) if frequencies[x] < 80])[0]
    widths = widths[upper:lower]
    
    wavelet_coeffs, freqs = pywt.cwt(y, widths, wavelet = wavelet, sampling_period=dt)
    t, _ = np.linspace(0, 200, wavelet_coeffs.shape[1], retstep=True)
    plt.imsave(DATA/'wavimgs'/f'{a.stem}-img.png', wavelet_coeffs[:,:1000], cmap='coolwarm')
    
    if plot_some:
        plt.imshow(wavelet_coeffs[:,:500], cmap='coolwarm')
        plt.show()

In [ ]:
get_wavelet_scalogram(example_fn, plot_some=True)

# Prepare the data: from WAV to spectograms

In [ ]:
if colab or gradient:
    !pip install fastaudio

In [ ]:
from fastaudio.core.all import *
from fastaudio.augment.all import *
from fastaudio.ci import skip_if_ci

In [ ]:
audio.ls()[0]

In [ ]:
at = AudioTensor.create(audio.ls()[0])

In [ ]:
at.show()

In [ ]:
?AudioConfig.BasicMelSpectrogram

In [ ]:
cfg = AudioConfig.BasicMelSpectrogram(n_fft=512, sample_rate=sr)
a2s = AudioToSpec.from_cfg(cfg)

In [ ]:
a2s.settings

In [ ]:
pipeline = Pipeline([AudioTensor.create, a2s])

a = AudioTensor.create(audio.ls()[0])
a.show()
pipeline(audio.ls()[0]).show()

# Split into train-val data

To use the same cross-validation setup as in original paper: 

From https://colab.research.google.com/github/fastaudio/fastaudio/blob/master/docs/ESC50:%20Environmental%20Sound%20Classification.ipynb

In [ ]:
def CrossValidationSplitter(col='fold', fold=1):
    "Split `items` (supposed to be a dataframe) by fold in `col`"
    def _inner(o):
        col_values = o.iloc[:,col] if isinstance(col, int) else o[col]
        valid_idx = (col_values == fold).values.astype('bool')
        return IndexSplitter(mask2idxs(valid_idx))(o)
    return _inner

# Data loaders

**Datablock:**

In [ ]:
auds = DataBlock(blocks = (AudioBlock, CategoryBlock),  
                 get_x = ColReader("filename", pref=audio), 
                 splitter = CrossValidationSplitter(fold=1),
                 batch_tfms = [a2s],
                 get_y = ColReader("category"))

**Dataloader:**

In [ ]:
dbunch = auds.dataloaders(df, bs=128)

In [ ]:
dbunch.show_batch(figsize=(10, 5))

### Training a model

In [ ]:
learn = cnn_learner(dbunch, 
            resnet34,
            n_in=1, 
            loss_func=CrossEntropyLossFlat(),
            metrics=[accuracy])

In [ ]:
lr = learn.lr_find()

In [ ]:
lr

In [ ]:
learn.fit_one_cycle(11, lr.lr_steep)

# Evaluate the results

Here are some predictions:

In [ ]:
learn.show_results()

Let's count up the mistakes:

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12))

Not too bad. How does this compare to other researchers working on the same data set? To answer that, we need to expand our evaluation somewhat as we should use the exact same data for evaluating our models.

## Comparing our results to those of others

Let's make a function that performs the steps above, but on different validation folds (i.e. the same cross-validation setup as in the original paper):

In [ ]:
def train_model(fold, auds=auds, epochs=12, bs=128, 
                item_tfms=None, n_in=1, callbacks = None):
    
    # Set up the data
    auds = DataBlock(blocks = (AudioBlock, CategoryBlock),  
                 get_x = ColReader("filename", pref=audio), 
                 splitter = CrossValidationSplitter(fold=fold),
                 item_tfms = item_tfms,
                 batch_tfms = [a2s],
                 get_y = ColReader("category"))
    dbunch = auds.dataloaders(df, bs=bs)
   
    # Create the model
    learn = cnn_learner(dbunch, resnet34, n_in=n_in, 
                        loss_func=CrossEntropyLossFlat(), 
                        metrics=[accuracy], cbs=callbacks
                       )
    
    # Find a learning rate
    lr = learn.lr_find(show_plot=False)
    
    # Train the model
    learn.fit_one_cycle(epochs, lr.lr_steep)
    
    # Return the accuracy of the model on the validation data
    return learn.metrics[0].value

The training process takes some time. We can load the outputs as images below. 

In [ ]:
# We train the model on all five folds specified by the creators of the data set
accs = []
for fold in df.fold.unique():
    accs.append(train_model(fold, epochs=10))

In [ ]:
from IPython.display import display, Image
display(Image('https://github.com/alu042/PCS956-DL-2021/raw/master/2-intro_to_practical_dl/assets/audio_init_crossval_1.png'))
display(Image('https://github.com/alu042/PCS956-DL-2021/raw/master/2-intro_to_practical_dl/assets/audio_init_crossval_2.png'))

In [ ]:
accs

In [ ]:
# Average accuracy across the folds:
np.array(accs).sum()/len(df.fold.unique())

In [ ]:
display(Image('https://github.com/alu042/PCS956-DL-2021/raw/master/2-intro_to_practical_dl/assets/audio_init_crossval_2.png'))

How does it compare to others? 

https://github.com/karolpiczak/ESC-50#results

We're at the level of [this poster](https://www.mi.t.u-tokyo.ac.jp/assets/publication/LEARNING_ENVIRONMENTAL_SOUNDS_WITH_END-TO-END_CONVOLUTIONAL_NEURAL_NETWORK-poster.pdf) submitted to the 2017 IEEE International Conference on Acoustics, Speech and Signal Processing (71%). Not too bad for such a simple setup, but, still, quite far from the top... 

# Improving our results

## Data augmentation

There are many approaches one could pursue for data augmentation in audio. From simple things like adding various forms of noise at random, resampling at random, or changing the pitch and speed, to more advanced ideas like [masking blocks of time-steps or frequencies](https://arxiv.org/abs/1904.08779).

Let's try a few of those propsed in the paper linked above. They come built-in to `fastaudio`.

In [ ]:
DBMelSpec = SpectrogramTransformer(mel=True, to_db=True)

In [ ]:
aud2spec = DBMelSpec(sample_rate=sr, n_mels=128, n_fft=512, hop_length=128)

In [ ]:
aud2spec.settings

In [ ]:
item_tfms = [aud2spec, MaskTime(size=4), MaskFreq(size=10)]

Here are some images showing the effect of the data augmentations (for the plots, we've made the data augmentation more pronounced for illustration purposes):

In [ ]:
auds.item_tfms = [aud2spec, MaskTime(size=30), MaskFreq(size=30)]
dls = auds.dataloaders(df, bs=128)
dls.show_batch(max_n=2, figsize=(8,8))

You notice the deleted temporal parts and the deleted frequencies.

Note: Again, the training takes some time and we can rather load the outputs from a previous round.

In [ ]:
# We train for a few additional epochs as we now have (in effect) more data
accs_da = []
for fold in df.fold.unique():
    accs_da.append(train_model(fold, item_tfms=item_tfms, epochs=15))

In [ ]:
accs_da

In [ ]:
# Average accuracy across the folds:
np.array(accs_da).sum()/len(df.fold.unique())

In [ ]:
from IPython.display import display, Image
display(Image('https://github.com/alu042/PCS956-DL-2021/raw/master/2-intro_to_practical_dl/assets/audio_da_crossval_1.png'))
display(Image('https://github.com/alu042/PCS956-DL-2021/raw/master/2-intro_to_practical_dl/assets/audio_da_crossval_2.png'))
display(Image('https://github.com/alu042/PCS956-DL-2021/raw/master/2-intro_to_practical_dl/assets/audio_da_crossval_3.png'))

A solid improvement over our previous result:

In [ ]:
# Our previous result:
np.array(accs).sum()/len(df.fold.unique())

## Adding additional images

Let's try to add _delta_ features as new images. They are computed using an estimate of the _derivatives_ of the first and second order in our signal. 

In [ ]:
item_tfms = [aud2spec, MaskTime(size=4), MaskFreq(size=10), Delta()]

Here's the effect of the Delta transformations:

In [ ]:
auds.item_tfms = item_tfms
dls = auds.dataloaders(df, bs=128)
dls.show_batch()

We now have three channels instead of one. The original image in channel 0, then the two images obtained from the (approximate) derivatives in the time and the frequency directions in channels 1 and 2.

Let's check if the information from the representations derived from Delta helps our models:

> The training takes a while. You can rather load the training outputs as images below.

In [ ]:
# We train for a few additional epochs as we now have (in effect) more data
accs_delta = []
for fold in df.fold.unique():
    accs_delta.append(train_model(fold, item_tfms=item_tfms, epochs=18, n_in=3))

In [ ]:
accs_delta

In [ ]:
# Average accuracy across the folds:
np.array(accs_delta).sum()/len(df.fold.unique())

In [ ]:
from IPython.display import display, Image
for fn in sorted(list((NB_DIR/'assets'/'audio_training').iterdir())):
    display(Image(fn))

This result is competitive with the 79.80% obtained in the paper [Look, Listen and Learn](https://openaccess.thecvf.com/content_iccv_2017/html/Arandjelovic_Look_Listen_and_ICCV_2017_paper.html) from ICCV2017 by the two Google DeepMind researchers Relja Arandjelović and Andrew Zisserman (who's also a relatively famous researcher in the VGG group at the University of Oxford). 

### Human performance

In the paper introducing the ECS-50 data set they reported an experiment where they crowdsourced labels from human listeners. The reported human accuracy was **81,30%** (in their somewhat limited experiment; read [the paper](https://www.karolpiczak.com/papers/Piczak2015-ESC-Dataset.pdf) for more details). 

### The state-of-the-art

https://github.com/YuanGongND/ast claims an accuracy of 95,70% on the ECS-50 data set. They used _Transformers_, which is increasingly common among many state of the art models in deep learning. We'll learn about transformers next week...

### An important warning / caveat

The ECS-50 evaluation setup suffers from a very common problem across many machine learning data sets used to benchmark and compare models. As _there is no test data that the researchers cannot access until their model's generalization performance is estimated_ it is very easy to fool yourself into thinking you have better results than you _actually_ have. 

As an example, here's the same model as above trained using what's called _early stopping_, i.e. stopping the training when the validation loss is smallest (or close to smallest). This approach is the same as picking the model from the epoch where the validation loss was smallest during the above training procedure. If we look at the accuracies we see that we would then be able to make the (invalid) accuracy claim of **80.6%**.

Another way to obtain invalid overestimates of the generalization performance would be to simply run the training above over and over, and then, because of randomness in many parts of the setup (e.g. randomness in initialization of the network weights, randomness in the data augmentation), find a cross-validation run that's better than the one we reported. 

You could also play with various hyperparameters of the model or the data augmentation setup, or simply try out a bunch of different number of epochs until you improve the results. 

> Again, I'm not recommending that you do this! It completely invalidates your results!

This issue, which is, again, quite common across machine learning data sets used in research, means that _it should be a requirement that all the source code needed to fully reproduce the results is published along with the papers_. 

> Note: even if the code is shared you should be vigilant about any sign of tinkering to obtain what seems to be better results but aren't. Like fixed random seeds using `random_state` or similar... Instabilities in the validation loss during the latter parts of the training process, meaning that it fluctuates significantly, can also be a worrying sign. 

# Your turn

> **Your turn!** Several of the tricks presented in the previous notebook can be applied. For example, would ensembling a few models improve the results? What about test-time augmentation?

> **Your turn!** Can you incorporate the wavelet images as another input to the model? Does it help?

The next two notebooks will present some additional ideas that you may want to consider applying also to this problem.